In [ ]:
import requests
import codecs
import urllib3
import datetime
import json
import pandas as pd
import base64


# Disable SSL warnings (since we are using verify=False)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# Configuration
REST_HOST = '100.66.135.17:8080'
MACAROON_PATH = '/Users/saurabhkumar/Desktop/Work/Lightning/lnnodefile/admin.macaroon'
TLS_PATH = '/Users/saurabhkumar/Desktop/Work/Lightning/lnnodefile/tls.cert'

headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}

In [ ]:
url = f'https://{REST_HOST}/v1/graph/info'
r = requests.get(url, headers=headers, verify=False)
print(r.json())

In [ ]:

def create_invoice(amount_sat, memo='lets roll'):
    url = f'https://{REST_HOST}/v1/invoices'
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    data = {
        'value': amount_sat,
        'memo': memo,
        'expiry': '3600'  # Expiry time in seconds
    }
    response = requests.post(url, headers=headers, json=data, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error creating invoice: {response.text}")
        return None

def decode_payment_request(payment_request):
    url = f"https://{REST_HOST}/v1/payreq/{payment_request}"
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    response = requests.get(url, headers=headers, verify=False)
    if response.status_code == 200:
        payment_data = response.json()
        # Extract payment_hash and return along with all payment data
        payment_hash = payment_data.get('payment_hash', None)  # Provide a default of None if not found
        return payment_hash, payment_data
    else:
        print(f"Error decoding payment request: {response.text}")
        # Return None for both values in case of an error
        return None, {}

def find_route(pub_key, amt_sat, amt_msat=None, final_cltv_delta=None, fee_limit=None,
               ignored_nodes=None, source_pub_key=None, use_mission_control=None,
               cltv_limit=None, outgoing_chan_id=None, last_hop_pubkey=None,
               time_pref=None):
    # Constructing the full URL with query parameters
    query_params = {
        'amt_msat': amt_msat,
        'final_cltv_delta': final_cltv_delta,
        'fee_limit': json.dumps(fee_limit) if fee_limit else None,
        'ignored_nodes': ','.join(ignored_nodes) if ignored_nodes else None,
        'source_pub_key': source_pub_key,
        'use_mission_control': str(use_mission_control).lower() if use_mission_control is not None else None,
        'cltv_limit': cltv_limit,
        'outgoing_chan_id': outgoing_chan_id,
        'last_hop_pubkey': base64.urlsafe_b64encode(bytes.fromhex(last_hop_pubkey)).decode() if last_hop_pubkey else None,
        'time_pref': time_pref
    }
    query_params = {k: v for k, v in query_params.items() if v is not None}
    query_string = '&'.join(f"{k}={v}" for k, v in query_params.items())
    url = f"https://{REST_HOST}/v1/graph/routes/{pub_key}/{amt_sat}?{query_string}"
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    
    response = requests.get(url, headers=headers, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error finding route: {response.text}")
        return None

def display_routes(routes):
    data = [{
        'Channel ID': hop['chan_id'],
        'Channel Details': f"{int(hop['chan_id']) >> 40}:{int(hop['chan_id']) >> 16 & 0xFFFFFF}:{int(hop['chan_id']) & 0xFFFF}",
        'Fee (msat)': hop['fee_msat'],
        'Pub Key': hop['pub_key'],
          } for route in routes for hop in route['hops']]
    df = pd.DataFrame(data)
    print(df)

def select_route(routes):
    valid_routes = [r for r in routes if len(set(hop['chan_id'] for hop in r['hops'])) > 1]
    return min(valid_routes, key=lambda x: sum(int(hop['fee_msat']) for hop in x['hops']), default=None)


def send_to_route_v2( route,payment_hash, skip_temp_err=False):
    """
    Send a payment through a specified route manually, useful for operations like rebalancing.

    Args:
        payment_hash (str): The payment hash of the invoice to pay, as a hexadecimal string.
        route (dict): A dictionary representing the route to be used for the payment.
        skip_temp_err (bool): Whether to skip temporary errors.

    Returns:
        dict: The JSON response from the LND node.
    """
    url = f'https://{REST_HOST}/v2/router/route/send'
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    data = {
        'payment_hash': base64.urlsafe_b64encode(bytes.fromhex(payment_hash)).decode(),
        'route': route,
        'skip_temp_err': skip_temp_err
    }

    response = requests.post(url, headers=headers, json=data, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error sending to route: {response.text}")
        return None

In [ ]:
# Assuming you have adjusted your probe_network function to handle the payment_hash
def probe_network(amount_sat, use_mission_control, outgoing_chan_id, last_hop_pubkey):
    invoice_response = create_invoice(amount_sat)
    if 'payment_request' in invoice_response:
        print("Invoice created successfully:", invoice_response['payment_request'])
        payment_hash, decoded_invoice = decode_payment_request(invoice_response['payment_request'])
        
        if payment_hash and decoded_invoice:
            print("Payment hash:", payment_hash)
            pub_key = decoded_invoice.get('destination')
            if pub_key:
                print("Destination public key:", pub_key)
                
                route_response = find_route(pub_key, amount_sat, use_mission_control=use_mission_control,
                                            outgoing_chan_id=outgoing_chan_id, last_hop_pubkey=last_hop_pubkey)
                if 'routes' in route_response and route_response.get('routes'):
                    print("Routes found. Displaying route information:")
                    display_routes(route_response['routes'])
                    
                    selected_route = select_route(route_response['routes'])
                    if selected_route:
                        print("Selected route based on criteria:")
                        display_routes([selected_route])
                        
                        payment_response = send_to_route_v2(selected_route, payment_hash, skip_temp_err=True)
                        print("Payment response:", payment_response)
                    else:
                        print("No suitable multi-hop route found.")
                else:
                    print("No route found or an error occurred.")
            else:
                print("Failed to extract destination public key.")
        else:
            print("Failed to decode payment request or payment hash missing.")
    else:
        print("Failed to create invoice or invalid invoice response.")

# Example usage
probe_network(
    amount_sat=50000,  # 2,000,000 satoshis
    use_mission_control=False,
    outgoing_chan_id="891194856252899329",
    last_hop_pubkey="03271338633d2d37b285dae4df40b413d8c6c791fbee7797bc5dc70812196d7d5c"
)


In [ ]:
def send_to_route_v2(payment_hash, route, skip_temp_err=False):
    """
    Send a payment through a specified route manually, useful for operations like rebalancing.

    Args:
        payment_hash (str): The payment hash of the invoice to pay, as a hexadecimal string.
        route (dict): A dictionary representing the route to be used for the payment.
        skip_temp_err (bool): Whether to skip temporary errors.

    Returns:
        dict: The JSON response from the LND node.
    """
    url = f'https://{REST_HOST}/v2/router/route/send'
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    data = {
        'payment_hash': base64.urlsafe_b64encode(bytes.fromhex(payment_hash)).decode(),
        'route': route,
        'skip_temp_err': skip_temp_err
    }

    response = requests.post(url, headers=headers, json=data, verify=TLS_PATH)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error sending to route: {response.text}")
        return None

# Example usage
route_example = {
    # Route details go here (this should be a properly formatted route dictionary)
}

payment_hash_example = "34735a45c034914b229f016f7cdeddb2dea9f81d205d2072ad94b498882fe89c"  # Example payment hash

response = send_to_route_v2(payment_hash_example, route_example, skip_temp_err=True)
print("Response from SendToRouteV2:", response)

In [ ]:
def send_payment_via_route(dest, amt_msat, payment_hash, final_cltv_delta, payment_request, timeout_seconds=60,
                           fee_limit_sat=None, outgoing_chan_id=None, cltv_limit=None, route_hints=None,
                           dest_custom_records=None, fee_limit_msat=None, last_hop_pubkey=None, allow_self_payment=None,
                           dest_features=None, max_parts=None, no_inflight_updates=None, outgoing_chan_ids=None,
                           payment_addr=None, max_shard_size_msat=None, amp=None, time_pref=None):
    """
    Sends a payment through the LND node using provided parameters.

    Parameters are expected to be in the appropriate format (e.g., hex strings where required).
    """
    url = f'https://{REST_HOST}/v2/router/send'
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    
    # Construct the data dictionary with required parameters
    data = {
        'dest': base64.urlsafe_b64encode(bytes.fromhex(dest)).decode(),
        'amt': str(amt_msat),
        'payment_hash': base64.urlsafe_b64encode(bytes.fromhex(payment_hash)).decode(),
        'final_cltv_delta': final_cltv_delta,
        'payment_request': payment_request,
        'timeout_seconds': timeout_seconds
    }

    # Optionally add additional parameters if they are provided
    optional_params = {
        'fee_limit_sat': fee_limit_sat,
        'outgoing_chan_id': outgoing_chan_id,
        'cltv_limit': cltv_limit,
        'route_hints': route_hints,
        'dest_custom_records': dest_custom_records,
        'fee_limit_msat': fee_limit_msat,
        'last_hop_pubkey': base64.urlsafe_b64encode(bytes.fromhex(last_hop_pubkey)).decode() if last_hop_pubkey else None,
        'allow_self_payment': allow_self_payment,
        'dest_features': dest_features,
        'max_parts': max_parts,
        'no_inflight_updates': no_inflight_updates,
        'outgoing_chan_ids': outgoing_chan_ids,
        'payment_addr': base64.urlsafe_b64encode(bytes.fromhex(payment_addr)).decode() if payment_addr else None,
        'max_shard_size_msat': max_shard_size_msat,
        'amp': amp,
        'time_pref': time_pref
    }

    # Filter out None values before adding to the data dictionary
    data.update({k: v for k, v in optional_params.items() if v is not None})

    # Make the API call
    response = requests.post(url, headers=headers, json=data, verify=TLS_PATH)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error sending payment: {response.text}")
        return None

# Example usage of send_payment_via_route with optional parameters
payment_response = send_payment_via_route(
    dest="0302a3b1cdef987654321...",  # Destination public key as hex
    amt_msat=500000,  # Amount in millisatoshis
    payment_hash="abcdef1234567890",  # Payment hash as hex
    final_cltv_delta=144,  # CLTV delta
    payment_request="lnbc500u1p...",  # Payment request string
    outgoing_chan_id="1234567890123456",  # Outgoing channel ID
    fee_limit_sat=1000,  # Fee limit in satoshis
    last_hop_pubkey="03271338633d2d37b285dae4df40b413d8c6c791fbee7797bc5dc70812196d7d5c"  # Last hop public key as hex
)
print("Payment response:", payment_response)

In [ ]:


def find_route(pub_key, amt_sat, amt_msat=None, final_cltv_delta=None, fee_limit=None,
               ignored_nodes=None, source_pub_key=None, use_mission_control=None,
               cltv_limit=None, outgoing_chan_id=None, last_hop_pubkey=None,
               time_pref=None):
    # Base URL with required path parameters
    url = f"https://{REST_HOST}/v1/graph/routes/{pub_key}/{amt_sat}"

    # Prepare the query string parameters
    query_params = {}
    if amt_msat:
        query_params['amt_msat'] = amt_msat
    if final_cltv_delta:
        query_params['final_cltv_delta'] = final_cltv_delta
    if fee_limit:
        query_params['fee_limit'] = json.dumps(fee_limit)  # Assuming fee_limit is a complex object
    if ignored_nodes:
        query_params['ignored_nodes'] = ','.join(ignored_nodes)
    if source_pub_key:
        query_params['source_pub_key'] = source_pub_key
    if use_mission_control is not None:
        query_params['use_mission_control'] = str(use_mission_control).lower()
    if cltv_limit:
        query_params['cltv_limit'] = cltv_limit
    if outgoing_chan_id:
        query_params['outgoing_chan_id'] = outgoing_chan_id
    if last_hop_pubkey:
        # Encode last_hop_pubkey in base64
        encoded_pubkey = base64.urlsafe_b64encode(bytes.fromhex(last_hop_pubkey)).decode()
        query_params['last_hop_pubkey'] = encoded_pubkey
    if time_pref:
        query_params['time_pref'] = time_pref

    # Encode query string
    query_string = '&'.join(f"{key}={value}" for key, value in query_params.items() if value is not None)
    full_url = f"{url}?{query_string}" if query_string else url

    # Set headers and make the request
    headers = {'Grpc-Metadata-macaroon': codecs.encode(open(MACAROON_PATH, 'rb').read(), 'hex')}
    print("Final URL:", full_url)  # Debugging output
    response = requests.get(full_url, headers=headers, verify=False)
    return response.json()

# Test the function
route_info = find_route(
    pub_key="0225c2a44be98ae923118e92c5b58033a0fd7dd3135be99ee1047ec9d30291dc5d",
    amt_sat=500000,
    use_mission_control=False,
    outgoing_chan_id="891194856252899329",
    last_hop_pubkey="03271338633d2d37b285dae4df40b413d8c6c791fbee7797bc5dc70812196d7d5c"
)

print("Route Info:", route_info)

if 'routes' in route_response and len(route_info['routes']) > 0:
    print("Routes found. Displaying route information:")
    display_routes(route_info['routes'])
    
#   lnmarket chan id  880174451395198976 kraken 890654996119158784    lqwd canada 891194856252899329

In [ ]:
                selected_route = select_route(route_info['routes'])
#                 selected_route = min(route_response['routes'], key=lambda x: sum(int(hop['fee_msat']) for hop in x['hops']))
                if selected_route:
                    print("Selected route based on criteria:")
                    display_routes([selected_route])

                    payment_response = send_payment_via_route(selected_route)
                    print("Payment response:", payment_response)

# from Claude

In [ ]:
import requests
import base64
import json
import os

# Step 1: Set up the necessary imports and configurations
LND_REST_ENDPOINT = "https://localhost:8080"  # Replace with your LND REST endpoint
MACAROON_PATH = "/path/to/your/admin.macaroon"  # Replace with your admin macaroon path
TLS_CERT_PATH = "/path/to/your/tls.cert"  # Replace with your TLS cert path

# Step 2: Define a function to make authenticated REST requests
def make_lnd_request(method, endpoint, data=None):
    url = f"{LND_REST_ENDPOINT}/v1/{endpoint}"
    
    with open(MACAROON_PATH, "rb") as f:
        macaroon = f.read().hex()
    
    headers = {
        "Grpc-Metadata-macaroon": macaroon,
    }
    
    response = requests.request(
        method,
        url,
        headers=headers,
        verify=TLS_CERT_PATH,
        data=json.dumps(data) if data else None
    )
    
    return response.json()

# Step 3: Implement a function to get node information
def get_node_info():
    return make_lnd_request("GET", "getinfo")

# Step 4: Create a function to list channels
def list_channels():
    return make_lnd_request("GET", "channels")

# Step 5: Develop a function to perform a probe (route estimation)
def probe_route(pub_key, amount_sat):
    data = {
        "pub_key": pub_key,
        "amt": str(amount_sat),
        "final_cltv_delta": 144
    }
    return make_lnd_request("POST", "channels/routes", data)

# Step 6: Write the main function to execute the probing process
def main():
    print("Getting node information...")
    node_info = get_node_info()
    print(f"Node pubkey: {node_info['identity_pubkey']}")
    
    print("\nListing channels...")
    channels = list_channels()
    for channel in channels.get("channels", []):
        print(f"Channel ID: {channel['chan_id']}, Remote pubkey: {channel['remote_pubkey']}")
    
    print("\nProbing routes...")
    target_pubkey = input("Enter the target node's public key: ")
    amount_sat = int(input("Enter the amount to probe (in satoshis): "))
    
    try:
        route = probe_route(target_pubkey, amount_sat)
        print("\nProbe successful! Route found:")
        for hop in route["routes"][0]["hops"]:
            print(f"Hop: {hop['pub_key']}, Channel: {hop['chan_id']}, Fee: {hop['fee']} msat")
        print(f"Total fees: {route['routes'][0]['total_fees']} msat")
        print(f"Total time lock: {route['routes'][0]['total_time_lock']}")
    except Exception as e:
        print(f"Probe failed: {str(e)}")

# Step 7: Run the script
if __name__ == "__main__":
    main()